In [18]:
import pandas as pd
import numpy as np

In [19]:
neg = pd.read_csv('negative.csv', sep=';', header=None)

In [20]:
pos = pd.read_csv('positive.csv', sep=';', header=None)
pos.columns = ['id', 'date', 'name', 'text', 'class', 'rep', 'retw', 'fav', 'count', 'foll', 'friend', 'list']
neg.columns = ['id', 'date', 'name', 'text', 'class', 'rep', 'retw', 'fav', 'count', 'foll', 'friend', 'list']

In [21]:
pos_choose = round(10000*0.7)#len(pos)
neg_choose = round(10000*0.3)#round(pos_choose*0.3/0.7)
#neg_choose = round(pos_choose*0.3/0.7)

pos_choose, neg_choose

(7000, 3000)

In [22]:
pos = pos.sample(frac=1).reset_index(drop=True)
neg = neg.sample(frac=1).reset_index(drop=True)

In [23]:
pos_new = pos[:pos_choose]
neg_new = neg[:neg_choose]


In [24]:
tweets = pos_new.append(neg_new)
tweets = tweets.sample(frac=1).reset_index(drop=True)

tweets.to_csv('tweets.csv', sep=';', encoding='utf-8')

In [25]:
tweets = pd.read_csv('tweets.csv', sep=';').drop('Unnamed: 0', axis = 1)

print("Null values: {0}".format(tweets.isnull().values.any()))
print("Count of NaN values: {0}".format(np.sum(tweets.isnull().values)))

    
print('Object types: ', [c for c in tweets.columns if tweets[c].dtype.name == 'object'])

Null values: False
Count of NaN values: 0
Object types:  ['name', 'text']


In [26]:
try:
    from nltk.corpus import stopwords
except:
    import nltk
    nltk.download('stopwords')
    from nltk.corpus import stopwords

stop_words = stopwords.words('russian')
removal = ['хорошо', 'лучше', 'нельзя', 'больше', 'никогда', 'ничего']

for i in removal:
    stop_words.remove(i)
stop_words.append('rt')

In [27]:
from re import compile, findall
from nltk import tokenize
from pymystem3 import Mystem

names = compile('@\S+')
exclude = compile('http:\S+')
tag = compile('#\S+')
emoji_good = compile('[:;]?-?[)d]+')
emoji_bad = compile('[:;]?-?[(\|]+')

m = Mystem()

new_tweets = []

list_tweets = tweets['text']

for num, elem in enumerate(list_tweets):
    elem = elem.lower()
    #print(elem)
    names_list = names.findall(elem)
    links = exclude.findall(elem)
    tags = tag.findall(elem)
    e_good = emoji_good.findall(elem)
    e_bad = emoji_bad.findall(elem)
    dots = findall('\.\.\.', elem)
    
    tweets.at[num, 'people'] = len(names_list)
    tweets.at[num, 'tags'] = len(tags)
    tweets.at[num, 'emoji_good'] = len(e_good)
    tweets.at[num, 'emoji_bad'] = len(e_bad)
    tweets.at[num, 'three_dots'] = len(dots)
    
    tweet = findall('\S+', elem)
    tweet = ' '.join([i for i in tweet if i not in names_list and i not in links and i not in tags])    
        
    tws = [k for k in tokenize.word_tokenize(tweet) if k not in stop_words and k.isalpha() == True and k not in e_good]    
    new_tweets.append(tws)
    if num%1000 == 0:
        print('done #' + str(num))
print('DONE ALL')  

done #0
done #1000
done #2000
done #3000
done #4000
done #5000
done #6000
done #7000
done #8000
done #9000
DONE ALL


In [28]:
new_tweets

[['бля',
  'ребят',
  'щас',
  'могу',
  'делать',
  'такую',
  'парашу',
  'которой',
  'ржу',
  'd',
  'завтра',
  'сделаю',
  'упорюсь',
  'сегодня'],
 ['телефоне', 'сидеть', 'выйдут', 'телеку', 'пожрать'],
 ['мищка', 'учил', 'такое', 'стихотворение', 'одно', 'мучилась', 'dd'],
 ['посмотрела',
  'голодные',
  'лучше',
  'б',
  'смотрела',
  'дико',
  'хочу',
  'третью',
  'часть'],
 ['успевайте', 'дактилоскопию', 'пока', 'бесплатно', 'добровольно'],
 ['это', 'нам', 'нужно', 'срочно', 'увидеться', 'прямо', 'приезжай'],
 ['сук', 'бесит', 'ничего', 'предъявишь'],
 ['привет', 'хах', 'думала', 'одна', 'вывожу'],
 ['оу', 'вещи', 'которых', 'лучше', 'узнавать'],
 ['позвонил',
  'apple',
  'online',
  'store',
  'оказалось',
  'межгород',
  'мной',
  'разговаривала',
  'ирландка'],
 ['всем', 'карме', 'говориться'],
 ['доделал',
  'прогу',
  'создания',
  'настроек',
  'девайса',
  'конечного',
  'пользователя',
  'больше',
  'возможностей'],
 ['сочувствую', 'мучился'],
 ['нормальный',
  'па

In [29]:
from gensim.models import Word2Vec, KeyedVectors

C:\programs\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [30]:
#model = KeyedVectors.load_word2vec_format('ruscorpora_upos_skipgram_300_5_2018.vec', binary=False)
my_model = Word2Vec(new_tweets, window=5, min_count=0, size=100, workers=4)

In [31]:
vectors = []
for i in new_tweets:
    vec = []
    for k in i:
        try:
            model = my_model.wv[k]
            vec.append(model)
        except:
            vec.append('no model')
            print('no model')
    vectors.append(vec)
print(len(vectors))

10000


In [32]:
sum_vec = []
for i in vectors:
    sum_vec.append(np.sum(i, axis=0)/len(i))

C:\programs\anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
for col in ['retw', 'count', 'foll', 'friend', 'list', 'people', 'tags', 'emoji_bad', 'emoji_good', 'three_dots']:
    x = tweets[col].values.reshape(-1, 1)
    standardized = scaler.fit_transform(x)
    tweets[col] = standardized

C:\programs\anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Колонки *id* и *namе* не несут никакой важной информации в связи с тем, что подавляющее большинство элементов в них является уникальным, формат колонки *date* нераспознаваем, поэтому ей сложно воспользоваться для обучения модели

In [34]:
try:
    tweets.drop('id', axis=1, inplace=True)
    tweets.drop('date', axis=1, inplace=True)
    tweets.drop('name', axis=1, inplace=True)
except:
    pass

In [35]:
f = ['f'+str(i) for i,l in enumerate(sum_vec[0])]

for i in f:
    tweets[i] = ""

In [36]:
for a,b in enumerate(sum_vec):
    sum_vec[a] = b.tolist()
print(len(sum_vec))
len(sum_vec[0])

10000


100

In [37]:
#vecs = sum_vec
#tweets_new = tweets.drop('text', axis=1)

for el, vecs in enumerate(sum_vec):
    if el%100 == 0:
        print('done vec #' + str(el))
    tweets.at[el, 'f0':'f99'] = vecs
    #for n, k in enumerate(f):
     #   tweets[k][el] = vecs[n]
      #  print(n)
tweets

#for string in range(len(tweets)):
 #   tweets.loc[string][f[0]:f[-1]] = sum_vec[string]
  #  break
#tweets

done vec #0
done vec #100
done vec #200
done vec #300
done vec #400
done vec #500
done vec #600
done vec #700
done vec #800
done vec #900
done vec #1000
done vec #1100
done vec #1200
done vec #1300
done vec #1400
done vec #1500
done vec #1600
done vec #1700
done vec #1800
done vec #1900
done vec #2000
done vec #2100
done vec #2200
done vec #2300
done vec #2400
done vec #2500
done vec #2600
done vec #2700
done vec #2800
done vec #2900
done vec #3000
done vec #3100
done vec #3200
done vec #3300
done vec #3400
done vec #3500
done vec #3600
done vec #3700
done vec #3800
done vec #3900
done vec #4000
done vec #4100
done vec #4200
done vec #4300
done vec #4400
done vec #4500
done vec #4600
done vec #4700
done vec #4800
done vec #4900
done vec #5000
done vec #5100
done vec #5200
done vec #5300
done vec #5400
done vec #5500
done vec #5600
done vec #5700
done vec #5800
done vec #5900
done vec #6000
done vec #6100
done vec #6200
done vec #6300
done vec #6400
done vec #6500
done vec #6600
done ve

,text,class,rep,retw,fav,count,foll,friend,list,people,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,"бля, ребят, я щас могу делать такую парашу, с ...",1,0,-0.076643,0,-0.004867,0.122934,1.084662,-0.069715,-0.780502,...,0.000484458,0.00260149,0.00139445,-0.00154277,0.00351553,-0.00245159,-0.00132584,0.0041037,-0.000112935,0.00376461
1,"И в телефоне сидеть только тогда, когда они вы...",-1,0,-0.076643,0,-0.336539,-0.068135,-0.118028,-0.094005,-0.780502,...,0.00122952,-0.00107383,-0.000908368,0.00132349,-0.00216294,0.000195872,-0.00111249,0.00126648,-0.00052307,0.00264134
2,"`Азаза, Мищка учил такое же стихотворение что ...",1,0,-0.076643,0,-0.198089,-0.053819,-0.136234,-0.094005,-0.780502,...,0.0017827,0.00197328,0.00115114,-0.00158276,0.00247113,0.000663884,0.000934864,0.000656162,0.000410851,1.84328e-05
3,"посмотрела голодные игры2,лучше б не смотрела(...",-1,0,-0.076643,0,-0.291181,-0.068135,-0.188093,-0.094005,-0.780502,...,0.00123022,0.00388989,8.62638e-05,-0.0010515,0.00265553,-0.00235312,-0.000841009,0.00213291,-0.00103786,0.0024314
4,"RT @dikifevaximo: Успевайте на дактилоскопию, ...",1,0,-0.040980,0,-0.323676,-0.053098,-0.114166,-0.094005,0.341876,...,0.000587394,0.00166456,0.00105087,-0.00206685,0.00177847,-0.000983338,-0.000419724,0.00129741,7.95162e-05,0.00161409
5,Это я. Нам нужно срочно увидеться. Прямо сейча...,-1,0,-0.076643,0,-0.152685,-0.063093,-0.168784,-0.094005,-0.780502,...,-9.9526e-05,0.00286588,0.00355425,-0.000451377,0.00515279,-0.000463699,5.15637e-05,0.00174404,-0.00131878,0.00643232
6,Сук бесит) но ничего не предъявишь http://t.co...,1,0,-0.076643,0,-0.307406,-0.066605,-0.191403,-0.094005,-0.780502,...,-0.00093606,0.000772528,-0.00121219,-0.000397254,-0.000722154,-0.00277616,0.0016314,-0.00106674,-0.000962057,0.00161227
7,"@U1berto, привет! хах) я думала, что я одна не...",1,0,-0.076643,0,-0.336808,-0.069756,-0.207402,-0.094005,0.341876,...,-0.000539849,0.000803011,0.00192655,3.8432e-06,0.00143261,2.74766e-05,0.00208462,0.00344193,0.000307543,0.00294229
8,"Оу, есть вещи, о которых лучше не узнавать(",-1,0,-0.076643,0,0.658343,-0.060122,-0.167681,-0.053522,-0.780502,...,0.00241131,0.000663432,0.00121791,-0.00204785,0.00251585,-0.00116671,-0.00031233,0.00165083,-0.00136352,0.00225069
9,"Позвонил в apple online store, оказалось межго...",1,0,-0.076643,0,-0.303731,-0.062553,-0.059549,-0.077812,-0.780502,...,-0.00188887,0.00133784,0.00042542,0.000296433,0.00101683,0.000242635,0.00159734,0.00119364,0.000736823,-0.0001149


In [38]:
tweets.drop('text', axis=1, inplace=True)
tweets.to_csv('tweets_coords.csv', sep=';', encoding='utf-8')

In [39]:
t = pd.read_csv('tweets_coords.csv', sep=';')

In [40]:
t.drop('Unnamed: 0', axis=1, inplace=True)

In [41]:
y = t['class'].values
X = t.drop('class', axis=1)

In [42]:
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy='most_frequent', axis=0)

X[:] = imputer.fit_transform(X)

In [43]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()

for col in ['f'+str(i) for i in range (0,100)]:
    x = X[col].values.reshape(-1, 1)
    standardized = scaler.fit_transform(x)
    X[col] = standardized

In [44]:
X

,rep,retw,fav,count,foll,friend,list,people,tags,emoji_good,...,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99
0,0.0,-0.076643,0.0,-0.004867,0.122934,1.084662,-0.069715,-0.780502,-0.200806,0.931783,...,-0.016337,0.484174,0.410764,-0.712772,1.043651,-1.073546,-0.856967,1.365540,0.280078,0.969111
1,0.0,-0.076643,0.0,-0.336539,-0.068135,-0.118028,-0.094005,-0.780502,-0.200806,-1.146698,...,0.511673,-1.378789,-1.083426,1.329777,-1.896987,0.642745,-0.699615,-0.166486,0.005550,0.418603
2,0.0,-0.076643,0.0,-0.198089,-0.053819,-0.136234,-0.094005,-0.780502,-0.200806,-0.107457,...,0.903697,0.165747,0.252892,-0.741271,0.502802,0.946147,0.810336,-0.496045,0.630680,-0.866877
3,0.0,-0.076643,0.0,-0.291181,-0.068135,-0.188093,-0.094005,-0.780502,-0.200806,-1.146698,...,0.512168,1.137245,-0.438055,-0.362685,0.598291,-1.009708,-0.499396,0.301361,-0.339030,0.315708
4,0.0,-0.040980,0.0,-0.323676,-0.053098,-0.114166,-0.094005,0.341876,-0.200806,0.931783,...,0.056611,0.009258,0.187834,-1.086242,0.144099,-0.121711,-0.188692,-0.149788,0.408898,-0.084852
5,0.0,-0.076643,0.0,-0.152685,-0.063093,-0.168784,-0.094005,-0.780502,-0.200806,-1.146698,...,-0.430192,0.618189,1.812161,0.064974,1.891516,0.215159,0.158889,0.091382,-0.527066,2.276546
6,0.0,-0.076643,0.0,-0.307406,-0.066605,-0.191403,-0.094005,-0.780502,-0.200806,0.931783,...,-1.023022,-0.442897,-1.280562,0.103543,-1.150865,-1.283958,1.324040,-1.426369,-0.288291,-0.085744
7,0.0,-0.076643,0.0,-0.336808,-0.069756,-0.207402,-0.094005,0.341876,-0.200806,-0.107457,...,-0.742238,-0.427446,0.756023,0.389372,-0.035005,0.533578,1.658299,1.008202,0.561530,0.566094
8,0.0,-0.076643,0.0,0.658343,-0.060122,-0.167681,-0.053522,-0.780502,-0.200806,-1.146698,...,1.349175,-0.498196,0.296217,-1.072699,0.525960,-0.240588,-0.109488,0.041051,-0.557013,0.227146
9,0.0,-0.076643,0.0,-0.303731,-0.062553,-0.059549,-0.077812,-0.780502,-0.200806,-0.107457,...,-1.698251,-0.156351,-0.217993,0.597876,-0.250318,0.673060,1.298922,-0.205822,0.848873,-0.932222


In [45]:
print("Null values: {0}".format(X.isnull().values.any()))
print("Count of NaN values: {0}".format(np.sum(X.isnull().values)))

Null values: False
Count of NaN values: 0


In [46]:
#from sklearn import svm


#model_svm = svm.SVC(kernel='linear', C=10, gamma='auto', class_weight='balanced') 
#model_svm.fit(X, y)
#model_svm.score(X, y)
#predicted = model_svm.predict(X)

In [47]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
if type(X) != np.ndarray:
    X = X.values
if type(y) != np.ndarray:
    y = y.values
parameters = {'kernel':('linear',),'C':[1, 10], 'class_weight':('balanced',)}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ('linear',), 'C': [1, 10], 'class_weight': ('balanced',)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [48]:
clf.best_estimator_

SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [49]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True) 
#X = X.values

In [50]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

accuracy = []
precision = []
recall = []
f1 = []

iteration_index = 0
model_svm = clf.best_estimator_
target_names = ['neg', 'pos']

for train_indexes, test_indexes in kf.split(X,y):
    iteration_index+=1
    X_train = X[train_indexes]
    y_train = y[train_indexes]
    
    X_test = X[test_indexes]
    y_test = y[test_indexes]
    
    model_svm.fit(X_train, y_train)
    model_svm.score(X_train, y_train)
    predicted = model_svm.predict(X_test)
    
    current_acc = accuracy_score(y_test, predicted)
    current_prec = precision_score(y_test, predicted)
    current_rec = recall_score(y_test, predicted)
    current_f1 = f1_score(y_test, predicted)
    #print("Iteration #{0}: Accuracy : {1}, Precision : {2}, Recall : {3}, F1 : {4}".format(iteration_index, current_acc, current_prec, current_rec, current_f1))
    
    print("Iteration #{0}: \n".format(iteration_index), classification_report(y_test, predicted, target_names=target_names))
    
    accuracy.append(current_acc)
    precision.append(current_prec)
    recall.append(current_rec)
    f1.append(current_f1)

print("\nOverall: ")
print("\tAccuracy : {0}".format(np.mean(accuracy)))
print("\tPrecision : {0}".format(np.mean(precision)))
print("\tRecall : {0}".format(np.mean(recall)))
print("\tF1 : {0}".format(np.mean(f1)))

Iteration #1: 
              precision    recall  f1-score   support

        neg       0.96      0.97      0.97       581
        pos       0.99      0.98      0.99      1419

avg / total       0.98      0.98      0.98      2000

Iteration #2: 
              precision    recall  f1-score   support

        neg       0.97      0.98      0.97       605
        pos       0.99      0.99      0.99      1395

avg / total       0.98      0.98      0.98      2000

Iteration #3: 
              precision    recall  f1-score   support

        neg       0.98      0.97      0.98       608
        pos       0.99      0.99      0.99      1392

avg / total       0.98      0.98      0.98      2000

Iteration #4: 
              precision    recall  f1-score   support

        neg       0.97      0.97      0.97       598
        pos       0.99      0.99      0.99      1402

avg / total       0.98      0.98      0.98      2000

Iteration #5: 
              precision    recall  f1-score   support

      

In [51]:
from sklearn.neighbors import KNeighborsClassifier
#neigh = KNeighborsClassifier(n_neighbors=5)

parameters = {'n_neighbors':[1, 10]}
knn = KNeighborsClassifier()
clf_knn = GridSearchCV(knn, parameters)
clf_knn.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 10]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [52]:
accuracy = []
precision = []
recall = []
f1 = []

iteration_index = 0
neigh = clf_knn.best_estimator_
target_names = ['neg', 'pos']

for train_indexes, test_indexes in kf.split(X,y):
    iteration_index+=1
    X_train = X[train_indexes]
    y_train = y[train_indexes]
    
    X_test = X[test_indexes]
    y_test = y[test_indexes]
    
    neigh.fit(X_train, y_train) 
    predicted = neigh.predict(X_test)
    
    current_acc = accuracy_score(y_test, predicted)
    current_prec = precision_score(y_test, predicted)
    current_rec = recall_score(y_test, predicted)
    current_f1 = f1_score(y_test, predicted)
    #print("Iteration #{0}: Accuracy : {1}, Precision : {2}, Recall : {3}, F1 : {4}".format(iteration_index, current_acc, current_prec, current_rec, current_f1))
    print("Iteration #{0}: \n".format(iteration_index), classification_report(y_test, predicted, target_names=target_names))

    
    accuracy.append(current_acc)
    precision.append(current_prec)
    recall.append(current_rec)
    f1.append(current_f1)

print("\nOverall: ")
print("\tAccuracy : {0}".format(np.mean(accuracy)))
print("\tPrecision : {0}".format(np.mean(precision)))
print("\tRecall : {0}".format(np.mean(recall)))
print("\tF1 : {0}".format(np.mean(f1)))

Iteration #1: 
              precision    recall  f1-score   support

        neg       0.96      0.69      0.80       602
        pos       0.88      0.99      0.93      1398

avg / total       0.90      0.90      0.89      2000

Iteration #2: 
              precision    recall  f1-score   support

        neg       0.97      0.69      0.81       593
        pos       0.88      0.99      0.93      1407

avg / total       0.91      0.90      0.90      2000

Iteration #3: 
              precision    recall  f1-score   support

        neg       0.97      0.71      0.82       542
        pos       0.90      0.99      0.94      1458

avg / total       0.92      0.91      0.91      2000

Iteration #4: 
              precision    recall  f1-score   support

        neg       0.97      0.69      0.81       621
        pos       0.88      0.99      0.93      1379

avg / total       0.91      0.90      0.89      2000

Iteration #5: 
              precision    recall  f1-score   support

      

Точность работы алгоритма получилась достаточно высокой, что удивительно (при использовании grid_search точность стала еще выше).
Это может быть связано с:
1. из-за недостоанчной мощности компьютера для ускорения обработки текста выборку пришлось сократить до 10000
2. модель w2v обучалась непосредственно на выборке, поэтому многие достаточно уникальные для выбрки слова и символы были достаточно точно выделены (к тому же обучение было без лемматизации в связи с недостоаточной мозностью компьютера). 
3. также были добавлены и другие признаки для каждого экземпляра выборки, что, возможно, позволило более точно описать каждый экземпляр